In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('cleaned_data\\cleaned_df.csv', parse_dates=['tstp'])

In [19]:
df.columns[-20:]

Index(['TRANSPORT_Travel To Work_Train / Tube / Tram',
       'TRANSPORT_Travel To Work_Work mainly at or from home', 'date', 'Type',
       'year', 'month', 'day', 'dayofweek_num', 'is_weekend', 'season',
       'no_work', 'hour', 'at_home', 'shifted_energy_24h',
       'shifted_energy_48h', 'shifted_energy_72h', 'shifted_energy_96h',
       'shifted_energy_120h', 'shifted_energy_144h', 'shifted_energy_168h'],
      dtype='object')

In [21]:
columns_desired = ['tstp', 'energy(kWh/hh)', 'LCLid', 'stdorToU', 'Acorn', 'visibility',
       'windBearing', 'temperature', 'dewPoint', 'pressure',
       'apparentTemperature', 'windSpeed', 'precipType', 'humidity',
       'summary', 'Type',
       'year', 'month', 'day', 'dayofweek_num', 'is_weekend', 'season',
       'no_work', 'hour', 'at_home', 'shifted_energy_24h',
       'shifted_energy_48h', 'shifted_energy_72h', 'shifted_energy_96h',
       'shifted_energy_120h', 'shifted_energy_144h', 'shifted_energy_168h']

sampled_df = df[columns_desired]

In [22]:
# get the first 80% and forecast the remaining 20%, must be in order

X = sampled_df.drop(['tstp', 'energy(kWh/hh)'], axis=1)
y = sampled_df['energy(kWh/hh)']


,tstp,energy(kWh/hh),LCLid,stdorToU,Acorn,visibility,windBearing,temperature,dewPoint,pressure,...,no_work,hour,at_home,shifted_energy_24h,shifted_energy_48h,shifted_energy_72h,shifted_energy_96h,shifted_energy_120h,shifted_energy_144h,shifted_energy_168h
0,2012-10-23 10:00:00,0.242,MAC003686,Std,ACORN-A,1.37,10.0,12.49,12.08,1024.14,...,0,10,0,0.224,0.422,0.261,1.050,0.279,0.224,0.317
1,2012-10-23 11:00:00,0.308,MAC003686,Std,ACORN-A,1.53,357.0,12.51,12.07,1024.36,...,0,11,0,0.242,0.608,0.838,0.828,0.202,0.215,0.801
2,2012-10-23 12:00:00,0.906,MAC003686,Std,ACORN-A,1.71,348.0,12.70,12.30,1024.48,...,0,12,0,0.231,0.400,0.787,2.461,0.203,0.213,0.557
3,2012-10-23 13:00:00,0.780,MAC003686,Std,ACORN-A,2.01,346.0,12.94,12.21,1024.48,...,0,13,0,0.207,0.284,0.222,0.267,0.258,0.228,0.548
4,2012-10-23 14:00:00,0.700,MAC003686,Std,ACORN-A,2.30,352.0,13.08,12.28,1024.14,...,0,14,0,0.224,0.359,0.228,0.447,0.247,0.209,0.549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755553,2014-01-31 20:00:00,0.076,MAC005344,Std,ACORN-P,8.85,177.0,7.08,5.47,990.60,...,0,20,1,0.037,0.095,0.088,0.403,0.284,0.312,0.409
755554,2014-01-31 21:00:00,0.037,MAC005344,Std,ACORN-P,8.96,181.0,7.53,6.06,989.18,...,0,21,1,0.039,0.047,0.040,0.384,0.255,0.734,0.528
755555,2014-01-31 22:00:00,0.039,MAC005344,Std,ACORN-P,8.79,182.0,7.68,6.66,987.81,...,0,22,1,0.040,0.041,0.040,0.423,0.384,0.275,0.662
755556,2014-01-31 23:00:00,0.041,MAC005344,Std,ACORN-P,6.07,192.0,8.31,7.23,986.83,...,0,23,1,0.042,0.040,0.060,0.365,0.420,0.283,0.386
